## EDA

In [10]:
#importing the libraries
import numpy as np
import pandas as pd
from scipy import stats
import datetime 
import plotly.express as px
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

In [13]:
#Adding scripts path
import sys, os
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from data_preprocessing import data_preProcessing_script
from cleaner import DataCleaner

In [14]:
store_data = pd.read_csv('../data/store.csv')
train_data = pd.read_csv('../data/train.csv')
test_data = pd.read_csv('../data/test.csv')